In [52]:
import pandas as pd
consumer_fraud = pd.read_csv('../data/raw/consumer_fraud_probability.csv')

In [58]:
import pandas as pd

df = pd.read_parquet('../test/')

In [59]:
df.head()

,user_id,merchant_abn,dollar_value,order_id,order_datetime
0,1,28000487688,133.226894,0c37b3f7-c7f1-48cb-bcc7-0a58e76608ea,2021-02-28
1,18485,62191208634,79.131400,9e18b913-0465-4fd4-92fd-66d15e65d93c,2021-02-28
2,1,83690644458,30.441348,40a2ff69-ea34-4657-8429-df7ca957d6a1,2021-02-28
3,18488,39649557865,962.813341,f4c1a5ae-5b76-40d0-ae0f-cb9730ac325a,2021-02-28
4,2,80779820715,48.123977,cd09bdd6-f56d-489f-81ea-440f4bda933c,2021-02-28


In [61]:
# Calculate the number of orders for each unique combination of user_id and order_datetime
user_order_counts = df.groupby(['user_id', 'order_datetime']).size().reset_index(name='num_orders')

# Calculate the highest dollar_value for each user_id
max_dollar_value = df.groupby(['user_id', 'order_datetime'])['dollar_value'].max().reset_index(name='max_dollar_value')

# Calculate the average dollar_value for each user_id
avg_dollar_value = df.groupby(['user_id', 'order_datetime'])['dollar_value'].mean().reset_index(name='avg_dollar_value')

# Merge the results with user_order_counts
user_order_counts = user_order_counts.merge(max_dollar_value, on=['user_id', 'order_datetime'], how='left')
user_order_counts = user_order_counts.merge(avg_dollar_value, on=['user_id', 'order_datetime'], how='left')

# Display the first few rows to verify
user_order_counts.head()

C:\Users\erikz\AppData\Local\Temp\ipykernel_1232\1215683969.py:2: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  user_order_counts = df.groupby(['user_id', 'order_datetime']).size().reset_index(name='num_orders')
C:\Users\erikz\AppData\Local\Temp\ipykernel_1232\1215683969.py:5: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  max_dollar_value = df.groupby(['user_id', 'order_datetime'])['dollar_value'].max().reset_index(name='max_dollar_value')
C:\Users\erikz\AppData\Local\Temp\ipykernel_1232\1215683969.py:8: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version

,user_id,order_datetime,num_orders,max_dollar_value,avg_dollar_value
0,1,2021-02-28,2,133.226894,81.834121
1,1,2021-03-01,0,NaN,NaN
2,1,2021-03-02,1,301.677498,301.677498
3,1,2021-03-03,0,NaN,NaN
4,1,2021-03-04,1,5.904951,5.904951


In [54]:
consumer_fraud.head()

,user_id,order_datetime,fraud_probability
0,6228,2021-12-19,97.629808
1,21419,2021-12-10,99.247380
2,5606,2021-10-17,84.058250
3,3101,2021-04-17,91.421921
4,22239,2021-10-19,94.703425


In [85]:
merged_fraud = user_order_counts.merge(consumer_fraud, on=['user_id', 'order_datetime'], how='inner')
merged_fraud.head()

,user_id,order_datetime,num_orders,max_dollar_value,avg_dollar_value,fraud_probability
0,1,2022-02-20,1,2479.076338,2479.076338,9.805431
1,2,2021-08-30,3,1665.505861,710.667418,9.599514
2,2,2021-09-25,1,2236.461666,2236.461666,10.069851
3,3,2021-11-03,1,2334.493717,2334.493717,8.300636
4,4,2021-10-09,3,1791.908327,775.013087,9.633302


In [91]:
merged_fraud_1 = merged_fraud.drop(columns=['user_id','order_datetime'])
merged_fraud_1['fraud_probability'] = merged_fraud_1['fraud_probability'].apply(lambda x: 0 if x < 15 else 1)

merged_fraud_1 = merged_fraud_1.rename(columns={'fraud_probability': 'is_fraud'})

merged_fraud_1.head()

,num_orders,max_dollar_value,avg_dollar_value,is_fraud
0,1,2479.076338,2479.076338,0
1,3,1665.505861,710.667418,0
2,1,2236.461666,2236.461666,0
3,1,2334.493717,2334.493717,0
4,3,1791.908327,775.013087,0


In [92]:
merged_fraud_2 = merged_fraud.drop(columns=['user_id','order_datetime'])
merged_fraud_2['fraud_probability'] = merged_fraud_2['fraud_probability'].apply(lambda x: 0 if x < 20 else 1)

merged_fraud_2 = merged_fraud_2.rename(columns={'fraud_probability': 'is_fraud'})

merged_fraud_2.head()

,num_orders,max_dollar_value,avg_dollar_value,is_fraud
0,1,2479.076338,2479.076338,0
1,3,1665.505861,710.667418,0
2,1,2236.461666,2236.461666,0
3,1,2334.493717,2334.493717,0
4,3,1791.908327,775.013087,0


In [109]:
from sklearn.linear_model import LogisticRegression

# Assuming merged_fraud_1 and merged_fraud_2 are already defined DataFrames

# Define the features and target variable for merged_fraud_1
X1 = merged_fraud_1.drop(columns=['is_fraud'])  # Drop the target column
y1 = merged_fraud_1['is_fraud']

# Define the features and target variable for merged_fraud_2
X2 = merged_fraud_2.drop(columns=['is_fraud'])  # Drop the target column
y2 = merged_fraud_2['is_fraud']

# Initialize the linear regression model
model1 = LogisticRegression()
model2 = LogisticRegression()

# Fit the model on the entire dataset for merged_fraud_1
model1.fit(X1, y1)

# Fit the model on the entire dataset for merged_fraud_2
model2.fit(X2, y2)

LogisticRegression()

In [115]:
merged_fraud_3 = merged_fraud.drop(columns=['user_id','order_datetime'])

In [117]:
from sklearn.linear_model import LinearRegression


X3 = merged_fraud_3.drop(columns=['fraud_probability'])
y3 = merged_fraud_3['fraud_probability']


# Initialize the linear regression model
model3 = LinearRegression()

# Fit the model on the entire dataset for merged_fraud_1
model3.fit(X3, y3)

LinearRegression()

In [99]:
merged = user_order_counts.merge(merged_fraud[['user_id', 'order_datetime']], on=['user_id', 'order_datetime'], how='left', indicator=True)


test_data = merged[merged['_merge'] == 'left_only'].drop(columns=['_merge'])


test_data.head()

,user_id,order_datetime,num_orders,max_dollar_value,avg_dollar_value
0,1,2021-02-28,2,133.226894,81.834121
1,1,2021-03-01,0,NaN,NaN
2,1,2021-03-02,1,301.677498,301.677498
3,1,2021-03-03,0,NaN,NaN
4,1,2021-03-04,1,5.904951,5.904951


In [100]:
test_data = test_data.dropna()
test_data = test_data.drop(columns=['user_id', 'order_datetime'])
test_data.head()

,num_orders,max_dollar_value,avg_dollar_value
0,2,133.226894,81.834121
2,1,301.677498,301.677498
4,1,5.904951,5.904951
6,1,18.135592,18.135592
9,1,64.670772,64.670772


In [118]:
# Predict using model1
model1_pred = model1.predict(test_data)

# Predict using model2
model2_pred = model2.predict(test_data)

# Print the predictions
print("Model 1 Predictions:", model1_pred)
print("Model 2 Predictions:", model2_pred)

Model 1 Predictions: [0 0 0 ... 0 0 0]
Model 2 Predictions: [0 0 0 ... 0 0 0]


In [119]:
model3_pred = model3.predict(test_data)

In [111]:
test_data_pred = test_data.copy()
test_data_pred['model1_pred'] = model1_pred
test_data_pred['model2_pred'] = model2_pred
test_data_pred.head()

,num_orders,max_dollar_value,avg_dollar_value,model1_pred,model2_pred
0,2,133.226894,81.834121,0,0
2,1,301.677498,301.677498,0,0
4,1,5.904951,5.904951,0,0
6,1,18.135592,18.135592,0,0
9,1,64.670772,64.670772,0,0


In [113]:
proportion_fraud_model1 = (model1_pred == 1).mean()
proportion_fraud_model2 = (model2_pred == 1).mean()

print("Proportion of instances classified as fraud by Model 1:", proportion_fraud_model1)
print("Proportion of instances classified as fraud by Model 2:", proportion_fraud_model2)

Proportion of instances classified as fraud by Model 1: 0.0011614601878376129
Proportion of instances classified as fraud by Model 2: 0.0005855387582820857


In [120]:
test_data_pred2 = test_data.copy()
test_data_pred2['model3_pred'] = model3_pred
test_data_pred2.head()

,num_orders,max_dollar_value,avg_dollar_value,model3_pred
0,2,133.226894,81.834121,6.411412
2,1,301.677498,301.677498,6.169553
4,1,5.904951,5.904951,5.547790
6,1,18.135592,18.135592,5.573501
9,1,64.670772,64.670772,5.671325
